https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

Sequence to sequence learning is about training models to convert sequences from one domain to sequences in another domain. This can be used for machine translation or for free-form question answering or in general, it is applicable any time you need to generate text

There are multiple ways to handle this task, either using RNNs or using 1D convnets.

# Trivial case - Input and output sequences have same length

When both input sequences and output sequences have the same length, you can implement such models simply with a keras LSTM or GRU layer. (or a stack)

Teaching a RNN to learn to add numbers, encoded as character strings

Input "535+51"
Output "596"

In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np

Using TensorFlow backend.


In [2]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [4]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
INVERT = True

In [5]:
# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

In [6]:
# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if INVERT:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [8]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [9]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

In [10]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

In [11]:
print('Training Data:')
print(x_train.shape)
print(y_train.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)


In [12]:
print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [13]:
from keras.layers import LSTM

In [14]:
model = Sequential()
# Encode the input sequence using LSTM
model.add(LSTM(128, input_shape = (MAXLEN, len(chars))))
# As the decoder RNN's input,
# repeatedly provide with the last hidden state of RNN for each time step
# DIGITS+1 as that's the maximum length of the output, e.g. max output is 999+999 = 1998
model.add(layers.RepeatVector(DIGITS + 1))
# By setting return_sequences to True, return not only the last output but
# all the outputs so far in the form of (num_samples, timesteps, output_dim).
model.add(LSTM(128, return_sequences = True))
# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))

In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [17]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)  
from IPython.display import FileLink
FileLink('model.png')

/home/sainath/code/py-snippets/nlp/model.png

In [18]:
model.fit(x_train, y_train, batch_size=128, epochs=25, verbose=2, validation_data=(x_val, y_val))

Train on 45000 samples, validate on 5000 samples
Epoch 1/25
21s - loss: 1.8823 - acc: 0.3213 - val_loss: 1.8066 - val_acc: 0.3407
Epoch 2/25
21s - loss: 1.7342 - acc: 0.3589 - val_loss: 1.6608 - val_acc: 0.3816
Epoch 3/25
21s - loss: 1.5830 - acc: 0.4108 - val_loss: 1.5116 - val_acc: 0.4394
Epoch 4/25
21s - loss: 1.4168 - acc: 0.4695 - val_loss: 1.3361 - val_acc: 0.5123
Epoch 5/25
21s - loss: 1.2709 - acc: 0.5310 - val_loss: 1.2195 - val_acc: 0.5540
Epoch 6/25
21s - loss: 1.1710 - acc: 0.5687 - val_loss: 1.1314 - val_acc: 0.5848
Epoch 7/25
21s - loss: 1.0860 - acc: 0.6013 - val_loss: 1.0471 - val_acc: 0.6206
Epoch 8/25
21s - loss: 0.9999 - acc: 0.6351 - val_loss: 0.9636 - val_acc: 0.6505
Epoch 9/25
21s - loss: 0.9059 - acc: 0.6735 - val_loss: 0.8693 - val_acc: 0.6865
Epoch 10/25
21s - loss: 0.8309 - acc: 0.7041 - val_loss: 0.8064 - val_acc: 0.7115
Epoch 11/25
21s - loss: 0.7553 - acc: 0.7354 - val_loss: 0.7377 - val_acc: 0.7346
Epoch 12/25
21s - loss: 0.6618 - acc: 0.7709 - val_loss: 0

# General case: different input and output sequence lengths

We will implement a basic character level sequence to sequence model. We apply it to translating short english sentences into short french sentences, character by character.

- We start with input sequences from a domain and corresponding target sequences from another domain
- An encoder LSTM turns input sequences to 2 state vectors (we keep the last LSTM state and discard the outputs)
- A decoder LSTM is trained to turn the target sequences into the same sequence but offset by one timestep in the future, a training process called 'teacher forcing' in this context. It uses as initials state the state vectors from the encoder. Effectively, the decoder learns to generate  targets[t+1...] given targets[...t], conditioned on the input sequence
- In the inference mode, we
- Encode the input sequence into state vectors
- Start with target sequence of size 1
- Feed the state vectors and 1-char target sequence to the decoder to produce predictions for the next character
- Sample the next character using these predictions
- Append the sampled character to the target sequence
- Repeat until we generate the end-of-sequence character or we hit the character limit

In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

Using TensorFlow backend.


In [2]:
num_samples = 10000 # number of samples to train on

In [3]:
# Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open('./fra.txt').read().split('\n')
for line in lines[:min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # lets use tab as start sequence character and \n as the end sequence character
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
input_texts[:5]

['Go.', 'Run!', 'Run!', 'Wow!', 'Fire!']

In [5]:
target_texts[:5]

['\tVa !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tÇa alors\u202f!\n',
 '\tAu feu !\n']

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [7]:
max_encoder_seq_length

16

In [8]:
max_decoder_seq_length

59

In [9]:
input_token_index = dict([(char, i) for i,char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [10]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype = 'float32'
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype = 'float32'
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype = 'float32'
)

In [11]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_target_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [12]:
encoder_inputs = Input(shape = (None, num_encoder_tokens))
encoder = LSTM(256, return_state = True)
encoder_ouputs, state_h, state_c = encoder(encoder_inputs)
# we discard the output, and only keep the states
encoder_states = [state_h, state_c]

In [13]:
decoder_inputs = Input(shape = (None, num_decoder_tokens))
decoder_lstm = LSTM(256, return_sequences = True, return_state = True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [14]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [15]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, 71)      0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, None, 93)      0                                            
____________________________________________________________________________________________________
lstm_1 (LSTM)                    [(None, 256), (None,  335872      input_1[0][0]                    
____________________________________________________________________________________________________
lstm_2 (LSTM)                    [(None, None, 256), ( 358400      input_2[0][0]                    
                                                                   lstm_1[0][1]            

In [18]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size = 64,
          verbose = 2,
          epochs = 25,
          validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/25
66s - loss: 0.9291 - val_loss: 0.9799
Epoch 2/25
65s - loss: 0.7422 - val_loss: 0.8107
Epoch 3/25
66s - loss: 0.6306 - val_loss: 0.7294
Epoch 4/25
65s - loss: 0.5723 - val_loss: 0.6764
Epoch 5/25
65s - loss: 0.5323 - val_loss: 0.6465
Epoch 6/25
66s - loss: 0.4996 - val_loss: 0.6043
Epoch 7/25
66s - loss: 0.4724 - val_loss: 0.5884
Epoch 8/25
66s - loss: 0.4490 - val_loss: 0.5708
Epoch 9/25
65s - loss: 0.4297 - val_loss: 0.5610
Epoch 10/25
65s - loss: 0.4123 - val_loss: 0.5415
Epoch 11/25
65s - loss: 0.3970 - val_loss: 0.5306
Epoch 12/25
66s - loss: 0.3817 - val_loss: 0.5248
Epoch 13/25
65s - loss: 0.3683 - val_loss: 0.5152
Epoch 14/25
65s - loss: 0.3559 - val_loss: 0.5072
Epoch 15/25
65s - loss: 0.3442 - val_loss: 0.5010
Epoch 16/25
66s - loss: 0.3331 - val_loss: 0.4980
Epoch 17/25
65s - loss: 0.3229 - val_loss: 0.4919
Epoch 18/25
65s - loss: 0.3127 - val_loss: 0.4907
Epoch 19/25
65s - loss: 0.3033 - val_loss: 0.4879
Epoch 20/25

Next: inference mode (sampling). Here's the drill:
1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token
3. Repeat with the current target token and current states

In [20]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [21]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
(i, char) for char, i in target_token_index.items())

In [22]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [23]:
for seq_index in range(100):
    # Take one sequence (part of the training test)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Allez chercher un manteille !

-
Input sentence: Run!
Decoded sentence: Toui le monde !

-
Input sentence: Run!
Decoded sentence: Toui le monde !

-
Input sentence: Wow!
Decoded sentence: Fais-le de coup de mainte.

-
Input sentence: Fire!
Decoded sentence: Donne-moi un bisa !

-
Input sentence: Help!
Decoded sentence: Appelle Tom !

-
Input sentence: Jump.
Decoded sentence: Laissez-moi manner.

-
Input sentence: Stop!
Decoded sentence: Arrête de tortir !

-
Input sentence: Stop!
Decoded sentence: Arrête de tortir !

-
Input sentence: Stop!
Decoded sentence: Arrête de tortir !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: I see.
Decoded sentence: Je l'ai vu.

-
Input sentence: I try.
Decoded sentence: Je l'ai contrarié.

-
Input sentence: I won!
Decoded sentence: J'ai été pruvent.

-
Input sentence: I won!
Decoded sentence: J'ai été pruvent.

-
Input sentence: Oh no!